# Utilities
> Build utilities for flask

In [ ]:
# default_exp utils

In [7]:
# export
import os
from pathlib import Path
import json
import pandas as pd

In [5]:
# export
HOME = Path(os.environ["HOME"])
EMBER = HOME/".torchember"
DATA = EMBER/"data"
LOG = EMBER/"LOG"
def check_existance():
    if DATA.exists()==False:
        return False
    if LOG.exists()==False:
        return False
    else:
        return True
    

Check if the data file exists, if not, use torch ember in your pytorch modeling first according to the short tutorial

In [6]:
check_existance()

True

In [14]:
# export
def get_ember_list():
    if check_existance()== False:
        return None
    else:
        return  list(i for i in os.listdir(DATA) if i[:5]=="base_")

In [29]:
get_ember_list()[:5]

['base_AlexNet_20200211_230327.json',
 'base_AlexNet_20200211_192928.json',
 'base_AlexNet_20200209_183014.json',
 'base_AlexNet_20200211_193437.json',
 'base_AlexNet_20200209_200003.json']

In [52]:
# export
def unpack_meta(fname):
    f = open(DATA/fname,"r")
    dict_ = json.loads(f.read())
    f.close()
    dict_["name"] = fname[5:-5]
    return dict_

def get_ember_df(ember_list):
    df = pd.DataFrame(list(unpack_meta(i) for i in ember_list))
    df = df.sort_values(by = "start",ascending = False)
    return df.reset_index().drop("index",axis=1)

In [53]:
df = get_ember_df(get_ember_list())
df.head(5)

,start,user,name
0,2020-02-11 23:07:43,salvor,AlexNet_20200211_230743
1,2020-02-11 23:07:40,salvor,AlexNet_20200211_230740
2,2020-02-11 23:06:14,salvor,AlexNet_20200211_230614
3,2020-02-11 23:06:09,salvor,AlexNet_20200211_230609
4,2020-02-11 23:04:37,salvor,AlexNet_20200211_230437


### Ember Reader
A reading handler to process Torch Ember data

In [12]:
# export
from torchember.core import emberTracker

class emberReader(object):
    def __init__(self, name):
        self.name = name
        self.t = emberTracker(name)
        self.structure = self.t[f"structure_{self.name}"]
        self.base = self.t[f"base_{self.name}"]

In [18]:
er = emberReader("AlexNet_20200211_230743")

In [19]:
er.structure

{'name': 'model(AlexNet)',
 'short': 'model(AlexNet)',
 'children': [{'name': 'model(AlexNet).features(Sequential)',
   'short': 'features(Sequential)',
   'children': [{'name': 'model(AlexNet).features(Sequential).0(Conv2d)',
     'short': '0(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).1(ReLU)',
     'short': '1(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).2(MaxPool2d)',
     'short': '2(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).3(Conv2d)',
     'short': '3(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).4(ReLU)',
     'short': '4(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).5(MaxPool2d)',
     'short': '5(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).6(Conv2d)',
     'short': '6(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).7(ReLU)',
     'short': '7(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).8(Conv2d)',
     'short': '8(Conv2d)'},
    {'name': 'model(AlexN

In [20]:
er.base

{'start': '2020-02-11 23:07:43', 'user': 'salvor'}

In [21]:
er.t.latest_df

,num,row_min,module,ts,ttype,tname,row_max
0,301056,"[-1.9999666213989258, -1.999962568283081]",model(AlexNet),2020-02-11 23:17:25,input,x,NaN
1,301056,"[-1.9999666213989258, -1.999962568283081]",model(AlexNet).features(Sequential),2020-02-11 23:17:25,input,input,NaN
2,301056,"[-1.9999666213989258, -1.999962568283081]",model(AlexNet).features(Sequential).0(Conv2d),2020-02-11 23:17:25,input,input,NaN
3,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-11 23:17:25,weight,weight_0,"[0.05245402455329895, 0.05220779776573181, 0.0..."
4,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-11 23:17:25,weight_grad,grad_0,"[3.595597809180617e-05, 3.1668449082644656e-05..."
...,...,...,...,...,...,...,...
75,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-11 23:17:26,weight,weight_1,"[-0.010682493448257446, 0.014477085322141647, ..."
76,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-11 23:17:26,weight_grad,grad_1,"[0.003000000026077032, 0.003000000026077032, 0..."
77,2000,"[-0.03675267845392227, -0.039796292781829834]",model(AlexNet).classifier(Sequential).6(Linear),2020-02-11 23:17:26,output,output_0,NaN
78,2000,"[-0.03675267845392227, -0.039796292781829834]",model(AlexNet).classifier(Sequential),2020-02-11 23:17:26,output,output_0,NaN
